# Dependencias

In [24]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
import plotly.io as pio

output.enable_custom_widget_manager()
pio.renderers.default = "colab"

# Coordenadas iniciales y matrices de transformación

In [25]:
# Coordinates relative to its father (local)
shoulder = np.array([[0,0,0,1]]).T
elbow = np.array([[4,0,0,1]]).T
wrist = np.array([[2,0,0,1]]).T
hand = np.array([[1,0,0,1]]).T

M_elbow = np.eye(4)
M_elbow[:,3] = elbow[:,0]

Ml_wrist = np.eye(4)
Ml_wrist[:,3] = wrist[:,0]
Mg_wrist = M_elbow @ Ml_wrist

Ml_hand = np.eye(4)
Ml_hand[:,3] = hand[:,0]
Mg_hand = Mg_wrist @ Ml_hand


M_elbow_base = M_elbow.copy()
Ml_wrist_base = Ml_wrist.copy()
Mg_wrist_base = Mg_wrist.copy()
Ml_hand_base = Ml_hand.copy()
Mg_hand_base = Mg_hand.copy()

elbow_t_x = 0
elbow_t_y = 90
elbow_t_z = 90

wrist_t_x = 0
wrist_t_y = 90
wrist_t_z = 90

hand_t_x = 0
hand_t_y = 90
hand_t_z = 90

# Funciones de transofrmación

In [26]:
def rotationMatrix(theta, axis):
    if axis == 'z':
      return np.array([
          [np.cos(theta), -1*np.sin(theta), 0, 0],
          [np.sin(theta), np.cos(theta), 0, 0],
          [0,0,1,0],
          [0,0,0,1]
      ])
    elif axis == 'x':
        return np.array([
            [1, 0, 0, 0],
            [0, np.cos(theta), -1*np.sin(theta), 0],
            [0, np.sin(theta), np.cos(theta), 0],
            [0,0,0,1]

        ])
    elif axis == 'y':
        return np.array([
            [np.cos(theta), 0, np.sin(theta), 0],
            [0,1,0,0],
            [-np.sin(theta),0,np.cos(theta),0],
            [0,0,0,1]
        ])
    else:
      raise Exception("Invalid axis")

def rotateShoulder(theta, axis):
    global M_elbow, Ml_wrist, Mg_wrist, Ml_hand, Mg_hand, elbow_t_x, elbow_t_y, elbow_t_z

    if axis == 'z':
      matrix = rotationMatrix(elbow_t_x, 'x') @ rotationMatrix(elbow_t_y, 'y') @ rotationMatrix(theta, 'z')
      elbow_t_z = theta
    elif axis == 'x':
      matrix = rotationMatrix(theta, 'x') @ rotationMatrix(elbow_t_y, 'y') @ rotationMatrix(elbow_t_z, 'z')
      elbow_t_x = theta
    elif axis == 'y':
      matrix = rotationMatrix(elbow_t_x, 'x') @ rotationMatrix(theta, 'y') @ rotationMatrix(elbow_t_z, 'z')
      elbow_t_y = theta
    else:
      raise Exception("Invalid axis")

    # Perform rotation
    M_elbow = matrix @ M_elbow_base

    # Update children
    Mg_wrist = M_elbow @ Ml_wrist
    Mg_hand = Mg_wrist @ Ml_hand

def rotateElbow(theta, axis):
    global M_elbow, Ml_wrist, Mg_wrist, Ml_hand, Mg_hand, wrist_t_x, wrist_t_y, wrist_t_z

    if axis == 'z':
      matrix = rotationMatrix(wrist_t_x, 'x') @ rotationMatrix(wrist_t_y, 'y') @ rotationMatrix(theta, 'z')
      wrist_t_z = theta
    elif axis == 'x':
      matrix = rotationMatrix(theta, 'x') @ rotationMatrix(wrist_t_y, 'y') @ rotationMatrix(wrist_t_z, 'z')
      wrist_t_x = theta
    elif axis == 'y':
      matrix = rotationMatrix(wrist_t_x, 'x') @ rotationMatrix(theta, 'y') @ rotationMatrix(wrist_t_z, 'z')
      wrist_t_y = theta
    else:
      raise Exception("Invalid axis")

    # Perform rotation
    Ml_wrist = matrix @ Ml_wrist_base
    Mg_wrist = M_elbow @ Ml_wrist
    # Update children
    Mg_hand = Mg_wrist @ Ml_hand


def rotateWrist(theta, axis):
    global M_elbow, Ml_wrist, Mg_wrist, Ml_hand, Mg_hand, hand_t_x, hand_t_y, hand_t_z

    if axis == 'z':
      matrix = rotationMatrix(hand_t_x, 'x') @ rotationMatrix(hand_t_y, 'y') @ rotationMatrix(theta, 'z')
      hand_t_z = theta
    elif axis == 'x':
      matrix = rotationMatrix(theta, 'x') @ rotationMatrix(hand_t_y, 'y') @ rotationMatrix(hand_t_z, 'z')
      hand_t_x = theta
    elif axis == 'y':
      matrix = rotationMatrix(hand_t_x, 'x') @ rotationMatrix(theta, 'y') @ rotationMatrix(hand_t_z, 'z')
      hand_t_y = theta
    else:
      raise Exception("Invalid axis")

    # Perform rotation
    Ml_hand = matrix @ Ml_hand_base
    Mg_hand = Mg_wrist @ Ml_hand



# Gráfico interactivo

In [ ]:
# @title
output.enable_custom_widget_manager()
points = np.zeros((4,4));
points[:,0] = np.array([0,0,0,1])
points[:,1] = M_elbow @ np.array([0,0,0,1])
points[:,2] = Mg_wrist @ np.array([0,0,0,1])
points[:,3] = Mg_hand @ np.array([0,0,0,1])

axis_length = 10

x_axis = go.Scatter3d(
    x=[-axis_length, axis_length],
    y=[0, 0],
    z=[0, 0],
    mode='lines',
    line=dict(width=6),
    showlegend=False
)

y_axis = go.Scatter3d(
    x=[0, 0],
    y=[-axis_length, axis_length],
    z=[0, 0],
    mode='lines',
    line=dict(width=6),
    showlegend=False
)

z_axis = go.Scatter3d(
    x=[0, 0],
    y=[0, 0],
    z=[-axis_length, axis_length],
    mode='lines',
    line=dict(width=6),
    showlegend=False
)

fig = go.FigureWidget(
    data=[
        go.Scatter3d(
            x=points[0, :],
            y=points[1, :],
            z=points[2, :],
            mode='markers+lines',
            marker=dict(size=6),
            line=dict(width=6)
        ),
        x_axis,
        y_axis,
        z_axis
    ],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(
                range=[-10, 10],
                autorange=False,
                showgrid=True,
                zeroline=False
            ),
            yaxis=dict(
                range=[-10, 10],
                autorange=False,
                showgrid=True,
                zeroline=False
            ),
            zaxis=dict(
                range=[-10, 10],
                autorange=False,
                showgrid=True,
                zeroline=False
            ),
            aspectmode='cube',
            bgcolor='white'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        height=500,
        width=500
    )
)

fig.layout.scene.xaxis.autorange = False
fig.layout.scene.yaxis.autorange = False
fig.layout.scene.zaxis.autorange = False

fig.layout.scene.xaxis.range = [-10, 10]
fig.layout.scene.yaxis.range = [-10, 10]
fig.layout.scene.zaxis.range = [-10, 10]


display(fig)

def update_points():
    points = np.zeros((4,4));
    points[:,0] = np.array([0,0,0,1])
    points[:,1] = M_elbow @ np.array([0,0,0,1])
    points[:,2] = Mg_wrist @ np.array([0,0,0,1])
    points[:,3] = Mg_hand @ np.array([0,0,0,1])

    with fig.batch_update():
        fig.data[0].x = points[0, :]
        fig.data[0].y = points[1, :]
        fig.data[0].z = points[2, :]

def update_shoulder(val, axis):
    theta = np.deg2rad(val)
    rotateShoulder(theta, axis)
    update_points()

def update_elbow(val, axis):
    theta = np.deg2rad(val)
    rotateElbow(theta, axis)
    update_points()

def update_wrist(val, axis):
    theta = np.deg2rad(val)
    rotateWrist(theta, axis)
    update_points()


# --- Shoulder ---
interact_sh_x = widgets.interact(
    lambda v: update_shoulder(v, 'x'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=0, description='Shoulder X')
)

interact_sh_y = widgets.interact(
    lambda v: update_shoulder(v, 'y'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=90, description='Shoulder Y')
)

interact_sh_z = widgets.interact(
    lambda v: update_shoulder(v, 'z'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=90, description='Shoulder Z')
)

# --- elbow ---
interact_el_x = widgets.interact(
    lambda v: update_elbow(v, 'x'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=0, description='Elbow X')
)

interact_el_y = widgets.interact(
    lambda v: update_elbow(v, 'y'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=90, description='Elbow Y')
)

interact_el_z = widgets.interact(
    lambda v: update_elbow(v, 'z'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=90, description='Elbow Z')
)

# --- Wrist ---
interact_wr_x = widgets.interact(
    lambda v: update_wrist(v, 'x'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=0, description='Wrist X')
)

interact_wr_y = widgets.interact(
    lambda v: update_wrist(v, 'y'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=90, description='Wrist Y')
)

interact_wr_z = widgets.interact(
    lambda v: update_wrist(v, 'z'),
    v=widgets.FloatSlider(min=-180, max=180, step=1, value=90, description='Wrist Z')
)


FigureWidget({
    'data': [{'line': {'width': 6},
              'marker': {'size': 6},
              'mode': 'markers+lines',
              'type': 'scatter3d',
              'uid': '0c611906-980f-4ffa-8325-b84981532417',
              'x': array([0., 4., 6., 7.]),
              'y': array([0., 0., 0., 0.]),
              'z': array([0., 0., 0., 0.])},
             {'line': {'width': 6},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '1b821261-be23-4fb2-b1fd-d2ee44980d9a',
              'x': [-10, 10],
              'y': [0, 0],
              'z': [0, 0]},
             {'line': {'width': 6},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '5d373e0f-0418-4426-872b-b89121a0d709',
              'x': [0, 0],
              'y': [-10, 10],
              'z': [0, 0]},
             {'line': {'width': 6},
              'mode': 'lines',
           

interactive(children=(FloatSlider(value=0.0, description='Shoulder X', max=180.0, min=-180.0, step=1.0), Outpu…

interactive(children=(FloatSlider(value=90.0, description='Shoulder Y', max=180.0, min=-180.0, step=1.0), Outp…

interactive(children=(FloatSlider(value=90.0, description='Shoulder Z', max=180.0, min=-180.0, step=1.0), Outp…

interactive(children=(FloatSlider(value=0.0, description='Elbow X', max=180.0, min=-180.0, step=1.0), Output()…

interactive(children=(FloatSlider(value=90.0, description='Elbow Y', max=180.0, min=-180.0, step=1.0), Output(…

interactive(children=(FloatSlider(value=90.0, description='Elbow Z', max=180.0, min=-180.0, step=1.0), Output(…

interactive(children=(FloatSlider(value=0.0, description='Wrist X', max=180.0, min=-180.0, step=1.0), Output()…

interactive(children=(FloatSlider(value=90.0, description='Wrist Y', max=180.0, min=-180.0, step=1.0), Output(…

interactive(children=(FloatSlider(value=90.0, description='Wrist Z', max=180.0, min=-180.0, step=1.0), Output(…